<a href="https://colab.research.google.com/github/tafsirnetlifyapp/colab/blob/main/Tools_Menghapus_Teks_Arab_Dan_Membiarkan_Terjemah_Tersusun_Rapih_Pada_JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Tools Menghapus Teks Arab Dan Membiarkan Terjemah Tersusun Rapih Pada JSON

import json
import re
from google.colab import files
import io

# --- 1. Upload File JSON ---
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# --- 2. Perbaiki JSON: Hilangkan koma terakhir jika ada ---
def fix_trailing_comma(json_str):
    json_str = re.sub(r',\s*}', '}', json_str)  # hapus koma terakhir sebelum }
    json_str = re.sub(r',\s*\]', ']', json_str)  # hapus koma terakhir sebelum ]
    return json_str

# --- 3. Baca dan parse JSON ---
with io.open(file_name, 'r', encoding='utf-8') as f:
    raw_text = f.read()
    fixed_text = fix_trailing_comma(raw_text)
    data = json.loads(fixed_text)

# --- 4. Fungsi hapus huruf Arab dan simbol non-Latin ---
def remove_non_latin(text):
    return re.sub(r'[^\x00-\x7F\u00C0-\u024F\u1E00-\u1EFFa-zA-Z0-9\s]', '', text)

# --- 5. Proses per ayat: hapus Arab + balik susunan kata ---
processed_data = {}
for ayah, text in data.items():
    parts = text.split(" | ")
    clean_parts = []
    for part in parts:
        cleaned = remove_non_latin(part).strip()
        if cleaned:
            clean_parts.append(cleaned)
    reversed_text = " | ".join(reversed(clean_parts))
    processed_data[ayah] = reversed_text

# --- 6. Simpan file hasil ---
output_filename = "latin_only_reversed_" + file_name
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=2)

# --- 7. Download file hasil ---
files.download(output_filename)
